#M5 FORECASTING KAGGLE COMPETITION

# 1.Importación de Datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# Lectura de  datos
train_sales = pd.read_csv('sales_train_validation.csv')#Continene los datos historicos de la venta diaria de cada uno de los productos
submission = pd.read_csv('sample_submission.csv') #Ejemplo del archivo que hay que mandar
calendar=pd.read_csv('calendar.csv')#Contiene información de días en los que se llevaron a cabo promociones y campañas
#sell_prices=pd.read_csv('sell_prices.csv')# Contiene información del precio de los productos por dia y por tienda

#MODELOS DE PRONÓSTICO


In [ ]:
# Definición de parámetros del modelo
n=365   # Se va a utilizar el último año de entrenamiento
v=30    # Se va a utilizar un mes de validación
s=0     # Para visualizar los resultados se va a tomar como ejemplo la serie 0

# Definición de datos de entrenamiento y de validación
d_cols = [c for c in train_sales.columns if 'd_' in c]
train_dataset = train_sales[d_cols[-n:-v]]
val_dataset = train_sales[d_cols[-v:]]
train_dataset_completo=train_sales[-n:]

In [ ]:
#Representación gráfico del dataset de entrenamiento y validación
fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(x=np.arange(n-v), mode='lines', y=train_dataset.loc[s].values, marker=dict(color="dodgerblue"), showlegend=False,name="Entrenamiento"),row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(n-v, n), y=val_dataset.loc[s].values, mode='lines', marker=dict(color="grey"), showlegend=False,name="Validación"),row=1, col=1) #,line=dict(dash="dot", width=3)
fig.update_layout(height=400, width=1200, title_text="Datos Entrenamiento-Validación de serie s",xaxis_title="N. Día",
    yaxis_title="Unidades Demandadas")

#### **Modelos Simples de Predicción:**  Naives-Simple Average-Media Móvil

In [ ]:
#Parámetros de los modelos
mm=30 #Periodos de Media Movil

pred_naives=[]
pred_simple_avg=[]
pred_moving_average=[]

# Ejecución de Modelos
for i in range(len(val_dataset.columns)):
    pred_naives.append(train_dataset[train_dataset.columns[-1]].values)
    pred_simple_avg.append(train_dataset.mean(axis=1))
    pred_moving_average.append(train_dataset.rolling(mm,axis=1).mean().iloc[:,-1])

# Organización de los resultados para submission       
pred_naives = np.transpose(np.array([row.tolist() for row in pred_naives]))
pred_simple_avg = np.transpose(np.array([row.tolist() for row in pred_simple_avg]))
pred_moving_average= np.transpose(np.array([row.tolist() for row in pred_moving_average]))

#### **Modelos de Predicción de Suavizamiento Exponencial:**  Exponential Smoothing, Holt, Holt Winters

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, Holt
pred_Holt = []
pred_Holt_Winters = []
pred_exp_smoothing=[]

#A continuación se lleva a cabo el proceso de predicción.
for row in (train_dataset[train_dataset.columns[-v:]].values) :                                 
    modelo_S = ExponentialSmoothing(row).fit(smoothing_level = 0.3, optimized=False)
    pred_exp_smoothing.append(modelo_S.forecast(v))
    modelo_Holt = Holt(row).fit(smoothing_level = 0.3, smoothing_slope = 0.01)
    pred_Holt.append(modelo_Holt.forecast(30)) 
    modelo_Holt_Winters = ExponentialSmoothing(row,seasonal_periods=7 ,trend='add', seasonal='add').fit()
    pred_Holt_Winters.append(modelo_Holt_Winters.forecast(v))
    
pred_exp_smoothing = np.array(pred_exp_smoothing).reshape((-1, v))
pred_Holt = np.array(pred_Holt).reshape((-1, 30))
pred_Holt_Winters=np.array(pred_Holt_Winters).reshape((-1, v))

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:924: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:930: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/

In [ ]:
# Hallar media histórica
#Overall_mean = train_dataset.mean(axis=1)
#Overall_mean=[Overall_mean]*len(train_dataset.columns)
#Overall_mean= np.transpose(np.array([row.tolist() for row in Overall_mean]))

In [ ]:
fig = make_subplots(rows=1, cols=1)
rolling_mean_30 = train_dataset.rolling(window=30,axis=1).mean()
mean_overall=train_dataset.rolling(window=len(train_dataset),axis=1).mean()

#Plot de datos de Entrenamiento
fig.add_trace(go.Scatter(x=np.arange(335), mode='lines',y=np.mean(train_dataset.values,axis=1), marker=dict(color="dodgerblue"), showlegend=False,name="Entrenamiento"),row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(335), mode='lines',y=rolling_mean_30.loc[s].values, marker=dict(color="coral"), showlegend=False,name="Entrenamiento"),row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(335), mode='lines',y=Overall_mean[s], marker=dict(color="yellow"), showlegend=False,name="Entrenamiento"),row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(335), mode='lines',y=pred_holt[s], marker=dict(color="red"), showlegend=False,name="Entrenamiento"),row=1, col=1)

#Plot de datos de Validación
fig.add_trace(go.Scatter(x=np.arange(335, 365), y=val_dataset.loc[s].values, mode='lines', marker=dict(color="Grey"), showlegend=False,name="Validación"),row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(335, 365), y=pred_naives[s], mode='lines', marker=dict(color="lightgreen"),name="Naives", showlegend= True),row=1, col=1) #,line=dict(dash="dot", width=3)
fig.add_trace(go.Scatter(x=np.arange(335, 365), y=pred_simple_avg[s], mode='lines', marker=dict(color="yellow"), name="Simple Average", showlegend= True ),row=1, col=1) #,line=dict(dash="dot", width=3)
fig.add_trace(go.Scatter(x=np.arange(335, 365), y=pred_moving_average[s], mode='lines', marker=dict(color="coral"), name="Moving Average 30 días", showlegend= True ),row=1, col=1) #,line=dict(dash="dot", width=3)
#fig.add_trace(go.Scatter(x=np.arange(335, 365), y=pred_exp_smoothing[s], mode='lines', marker=dict(color="pink"), name="Exponential Smoothing", showlegend= True ),row=1, col=1) #,line=dict(dash="dot", width=3)
fig.add_trace(go.Scatter(x=np.arange(335, 365), y=pred_Holt[s], mode='lines', marker=dict(color="red"), name="Holt", showlegend= True ),row=1, col=1) #,line=dict(dash="dot", width=3)
#fig.add_trace(go.Scatter(x=np.arange(335, 365), y=pred_Holt_Winters[s], mode='lines', marker=dict(color="Purple"), name="Holt Winters", showlegend= True ),row=1, col=1) #,line=dict(dash="dot", width=3)

fig.update_layout(height=400, width=1200, title_text="Predicción Modelos Básicos",xaxis_title="Día",
    yaxis_title="Unidades Demandadas")

NameError: ignored

#### **Modelos de Predicción ARIMA:** 

Parámetros:

*   p: Orde de Componente Autoregresiva
*   d: Número periodos que hacen ser
*   q: Orde de Componente de Media Móvil

p is the order of the AR term

q is the order of the MA term

d is the number of differencing required to make the time series stationary

In [ ]:
predictions = []
for row in tqdm(train_dataset[train_dataset.columns[-30:]].values[:3]):
    fit = sm.tsa.statespace.SARIMAX(row, seasonal_order=(0, 1, 1, 7)).fit()
    predictions.append(fit.forecast(30))
predictions = np.array(predictions).reshape((-1, 30))
error_arima = np.linalg.norm(predictions[:3] - val_dataset.values[:3])/len(predictions[0])

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
data1 = pd.Series(train_dataset.iloc[0].values)
modelo_ARIMA1 = ARIMA(data1, order=(5,1,0))
model_fit = modelo_ARIMA1.fit(disp=0)
print(model_fit.summary())
pred_ARIMA2=model_fit.forecast(30,dynamic=True)

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                  334
Model:                 ARIMA(5, 1, 0)   Log Likelihood                -464.487
Method:                       css-mle   S.D. of innovations              0.971
Date:                Tue, 31 Mar 2020   AIC                            942.974
Time:                        05:40:27   BIC                            969.652
Sample:                             1   HQIC                           953.610
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0028      0.015      0.189      0.850      -0.026       0.032
ar.L1.D.y     -0.7978      0.054    -14.894      0.000      -0.903      -0.693
ar.L2.D.y     -0.6228      0.065     -9.509      0.0

TypeError: ignored

In [ ]:
pred_ARIMA2

30     0.010110
31     0.002044
32     0.002183
33     0.001713
34     0.002051
         ...   
330    0.002826
331    0.002826
332    0.002826
333    0.002826
334    0.002826
Length: 305, dtype: float64

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, Holt
pred_Holt = []
pred_Holt_Winters = []
pred_exp_smoothing=[]

#A continuación se lleva a cabo el proceso de predicción.
for row in (train_dataset[train_dataset.columns[-v:]].values) :                                 
    modelo_S = ExponentialSmoothing(row).fit(smoothing_level = 0.3, optimized=False)
    pred_exp_smoothing.append(modelo_S.forecast(v))
    #modelo_Holt = Holt(row).fit(smoothing_level = 0.3, smoothing_slope = 0.01)
    #pred_Holt.append(modelo_Holt.forecast(30)) 
    #modelo_Holt_Winters = ExponentialSmoothing(row,seasonal_periods=7 ,trend='add', seasonal='add').fit()
    #pred_Holt_Winters.append(modelo_Holt_Winters.forecast(v))
    
#pred_exp_smoothing = np.array(pred_exp_smoothing).reshape((-1, v))
#pred_Holt = np.array(pred_Holt).reshape((-1, 30))
#pred_Holt_Winters=np.array(pred_Holt_Winters).reshape((-1, v))

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:924: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/holtwinters.py:930: RuntimeWarning:

divide by zero encountered in log



In [ ]:
pred_exp_smoothing[0]

array([1.17979733, 1.17979733, 1.17979733, 1.17979733, 1.17979733,
       1.17979733, 1.17979733, 1.17979733, 1.17979733, 1.17979733,
       1.17979733, 1.17979733, 1.17979733, 1.17979733, 1.17979733,
       1.17979733, 1.17979733, 1.17979733, 1.17979733, 1.17979733,
       1.17979733, 1.17979733, 1.17979733, 1.17979733, 1.17979733,
       1.17979733, 1.17979733, 1.17979733, 1.17979733, 1.17979733])

In [ ]:
predictions=[]
for row in train_dataset[train_dataset.columns[-30:]].values[:3]:
    fit = SARIMAX(row, seasonal_order=(0, 1, 1, 7)).fit()
    predictions.append(fit.forecast(30))
predictions = np.array(predictions).reshape((-1, 30))
#error_arima = np.linalg.norm(predictions[:3] - val_dataset.values[:3])/len(predictions[0])

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:993: UserWarning:

Non-invertible starting seasonal moving average Using zeros as starting parameters.



In [ ]:
predictions[0]

array([1.12659034, 1.48492581, 0.24796716, 1.24757808, 0.98170425,
       0.59994339, 1.80005165, 1.22512242, 1.49706074, 0.24946167,
       1.24776214, 0.98172692, 0.59994619, 1.80005199, 1.22512246,
       1.49706075, 0.24946167, 1.24776214, 0.98172692, 0.59994619,
       1.80005199, 1.22512246, 1.49706075, 0.24946167, 1.24776214,
       0.98172692, 0.59994619, 1.80005199, 1.22512246, 1.49706075])

In [ ]:
train_dataset[train_dataset.columns[-30:]][:1]

,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883
0,0,1,4,2,0,2,0,1,1,0,1,0,0,1,1,3,0,0,0,1,1,1,3,1,3,1,2,2,0,1


In [ ]:
from statsmodels.tsa.api import SARIMAX
pred_ARIMA = []

#A continuación se lleva a cabo el proceso de predicción.
#for row in (train_dataset[train_dataset.columns[-v:]].values) :                               
sarima = SARIMAX(train_dataset.iloc[0,-30:].values,order=(7,1,7),seasonal_order=(0,0,0,0),enforce_stationarity=False, enforce_invertibility=False).fit()
pred_ARIMA= sarima.forecast()
#pred_ARIMA = np.array(pred_ARIMA).reshape((-1, 30))

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [ ]:
pred_ARIMA[0]

-2.4710904308927484

In [ ]:
train_dataset.iloc[1,-30:].values


array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1])

In [ ]:
train_dataset.iloc[0,-30:]

d_1854    0
d_1855    1
d_1856    4
d_1857    2
d_1858    0
d_1859    2
d_1860    0
d_1861    1
d_1862    1
d_1863    0
d_1864    1
d_1865    0
d_1866    0
d_1867    1
d_1868    1
d_1869    3
d_1870    0
d_1871    0
d_1872    0
d_1873    1
d_1874    1
d_1875    1
d_1876    3
d_1877    1
d_1878    3
d_1879    1
d_1880    2
d_1881    2
d_1882    0
d_1883    1
Name: 0, dtype: int64

In [ ]:
fig.add_trace(go.Scatter(x=np.arange(335, 365), y=predictions[0], mode='lines', marker=dict(color="pink"), name="Exponential Smoothing", showlegend= True ),row=1, col=1) #,line=dict(dash="dot", width=3)


In [ ]:
# A continuaciín se reorganiza la tabla con el objetivo de que los días queden en filas
def melt_sales(df):
    df = df.drop(["item_id", "dept_id", "cat_id", "store_id", "state_id"], axis=1).melt(
        id_vars=['id'], var_name='d', value_name='demand')
    return df

train_sales_melt = melt_sales(train_sales)